In [1]:
import os
import pickle

import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [7]:
with open(os.path.join('../intermediate/output/', 'val.pkl'), 'rb') as f:
    target = (pickle.load(f) != 0).reshape(768, -1)

In [3]:
file_list = ['val_dn_31.pkl', 'val_tn_27.pkl', 'val_tn_31.pkl',
             'val_tn_41.pkl', 'val_unet_23.pkl', 'val_unet_27.pkl',
             'val_unet_33.pkl', 'val_unet_38.pkl', 'val_unet_41.pkl']
val_list = []
for file in file_list:
    with open(os.path.join('../intermediate/output/', file), 'rb') as f:
        val_list.append(pickle.load(f))

In [4]:
def dice_score(probs):
    m1    = probs.reshape(768, -1)
    score = (2*(m1*target).sum(axis=1))/(m1.sum(axis=1)+target.sum(axis=1))
    
    return score.sum()/768

In [5]:
def loss(weight=None, threshold=127.5):
    if weight is None:
        weight = [1.] * len(val_list)
     
    pred = 0.
    for i in range(len(val_list)):
        pred += weight[i] * val_list[i]
    pred = pred / sum(weight)
    pred = pred > threshold
    return dice_score(pred)

In [8]:
loss()

0.99718389423972376

In [22]:
weight=[0., 1., 1., 1., 1., 1., 1., 1., 1.]
loss(weight)

0.99718099918615399

In [26]:
weight=[1., 0., 1., 1., 1., 1., 1., 1., 1.]
loss(weight)

0.99718685666982321

In [27]:
weight=[1., 0., 0., 1., 1., 1., 1., 1., 1.]
loss(weight)

0.99717862316051198

In [28]:
weight=[1., 0., 1., 0., 1., 1., 1., 1., 1.]
loss(weight)

0.99717089369879941

In [29]:
weight=[1., 0., 1., 1., 0., 1., 1., 1., 1.]
loss(weight)

0.99719180696985255

In [30]:
weight=[1., 0., 1., 1., 0., 0., 1., 1., 1.]
loss(weight)

0.99718300274965266

In [31]:
weight=[1., 0., 1., 1., 0., 1., 0., 1., 1.]
loss(weight)

0.99718604870454863

In [25]:
weight=[0., 0., 1., 0., 0., 0., 0., 0., 1.]
loss(weight)

0.99710382792262886